<a href="https://colab.research.google.com/github/yuce-8/alpha/blob/main/YUCE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YUCE-8.tech API documentation
**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**

get some background information here: https://medium.com/swlh/forecasting-financial-data-78a6857860f2


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install requests

import json
import requests

## Connect to YUCE-8
this is subject to change in the next versions

In [ ]:
class GCF():
  def __init__(self):
    #self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_api'
    self.url = 'https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api'
    print('YUCE-8 ready @ ', self.url)

  def push(self, json_obj, return_result=False):
    r = requests.post(self.url, json=json_obj)
    if return_result:
      return r

yuce_8 = GCF()

YUCE-8 ready @  https://us-central1-yuce-8-v1.cloudfunctions.net/yuce_8_alpha_2_api


## Test connection
the test will return the current backend version

In [ ]:
def get_current_backend_version():
   req = yuce_8.push({'action': 'get_version'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']


print(get_current_backend_version())

v0.1.3 alpha


## Create a test key
Key can be reused for 50 calls

In [ ]:
def create_test_key(your_mail_address):
   req = yuce_8.push({'action': 'create_test_license', 'user_name': your_mail_address}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']



key = create_test_key('torsten@yuce-8.info')
print('our new key = ', key)

our new key =  a0ec44a0-47de-4de0-a012-2c141134c331


## List all available forecasts

In [ ]:
def get_available_forecasts():
   req = yuce_8.push({'action': 'get_available_forecasts'}, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']
   else:
     return ';-( ' + req.json()['msg']

get_available_forecasts()

[{'BTC-USD': ['1d'], '^NSEI': ['1d']}]

## Get latest forecast

In [ ]:
def get_latest_forecast(key, symbol, interval):
   req = yuce_8.push(
      {
        'action': 'get_forecast',
        'key': key,
        'symbol': symbol,
        'interval': interval
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg']['f_0']['yuce_8'], req.json()['msg']['api_calls_left_with_this_key'] 
   else:
     return ';-( ' + req.json()['msg'], -1

# get the forecast:
f_0, calls_left = get_latest_forecast(key, 'BTC-USD', '1d')

# print out the content:
if calls_left != -1:
  print('how many API calls are left?', calls_left)
  print('when was the forecast created?', f_0['forecast_created_at'])
  print('what is the last quote that was used to forecast?', f_0['last_quote'], ' @ ', f_0['last_date'])
  print('what is the name of the magic model that was used to forecast?', f_0['model_name'], 'last trained @', f_0['training_date'])
  print('what is the forecasted top range?', f_0['quotes_forecast_top'])
  print('what is the forecasted bottom range?', f_0['quotes_forecast_bottom'])

  # display the forecast timestep per timestep:
  print('timestep', '0 = ', f_0['last_quote'], ' @ ', f_0['last_date'])
  quote_0 = f_0['last_quote']
  top_range = f_0['quotes_forecast_top']
  bottom_range = f_0['quotes_forecast_bottom']

  for i in range(0, len(top_range)):
    forecast_top_at_i = top_range[i]
    forecast_bottom_at_i = bottom_range[i] 
    
    # see if it is a bearish or bullish forecast
    bearish = forecast_top_at_i <= quote_0

    print('timestep', i+1, 'should be w/in', ']' if bearish else '[', forecast_bottom_at_i, ', ', forecast_top_at_i, ']' if bearish else '[')

else:
  print(f_0)

how many API calls are left? 49
when was the forecast created? 2021-02-18 03:04:56.214622+01:00
what is the last quote that was used to forecast? 47504.8515625  @  2021-02-12 00:00:00
what is the name of the magic model that was used to forecast? model typ Y-3 last trained @ 2021-02-16 22:21:19
what is the forecasted top range? [48550.858926344124, 49776.87953492249, 49043.1741983894, 49043.1741983894, 49776.87953492249, 50928.711655411156, 50928.711655411156, 50928.711655411156, 49776.87953492249, 50928.711655411156, 49776.87953492249, 49776.87953492249, 50928.711655411156]
what is the forecasted bottom range? [48174.66996953125, 49043.1741983894, 48550.858926344124, 48550.858926344124, 49043.1741983894, 49776.87953492249, 49776.87953492249, 49776.87953492249, 49043.1741983894, 49776.87953492249, 49043.1741983894, 49043.1741983894, 49776.87953492249]
timestep 0 =  47504.8515625  @  2021-02-12 00:00:00
timestep 1 should be w/in [ 48174.66996953125 ,  48550.858926344124 [
timestep 2 sho

## Get backtesting statistics

In [ ]:
def get_backtest_statistics(key):
   req = yuce_8.push(
      {
        'action': 'get_backtest_statistics',
        'key': key
      }, return_result=True)
   if req.status_code == 200:
     return req.json()['msg'], req.json()['msg']['api_calls_left_with_this_key'] 
   else:
     return ';-( ' + req.json()['msg'], -1

# pull the backtesting results:
results, calls_left = get_backtest_statistics(key)
print(calls_left)
print(results)

48
{'api_calls_left_with_this_key': 48, 'key': 'a0ec44a0-47de-4de0-a012-2c141134c331', 'results': [{'confidence intervals 80%': [[1.0033615744792204, 1.0064157330311556], [1.0053238357284355, 1.009229472758503], [1.00728974014607, 1.0119124922147698], [1.0094844844004836, 1.0147424195149217], [1.0119669266223599, 1.0177509955883335], [1.0146650395115748, 1.0207930484095677], [1.0175062529228736, 1.0238500722266626], [1.0204227812434319, 1.0269116513584968], [1.0233353220738581, 1.029925242730219], [1.026177260821007, 1.0328277612279433], [1.0291350001883262, 1.0358259563851602], [1.0321719835756977, 1.0388874818326275], [1.0348144277370117, 1.0416130319797852]], 'confidence intervals 95%': [[1.002545968309904, 1.007231339200472], [1.0042808441473947, 1.0102724643395438], [1.0060552446074995, 1.0131469877533403], [1.008080364658736, 1.0161465392566693], [1.0104223040193816, 1.0192956181913118], [1.0130285685236295, 1.022429519397513], [1.015812150251642, 1.0255441748978942], [1.01868994

# DISCLAIMER

**THIS IS FOR EDUCATIONAL / RESEARCH & DEVELOPMENT PURPOSES ONLY!**


The information on this site is provided by YUCE-8 and the author and it is not to be construed as an offer or solicitation for the purchase or sale of any financial instrument or the provision of an offer to provide investment services. Information, opinions and comments contained on this site are not under the scope of investment advisory services. Investment advisory services are given according to the investment advisory contract, signed between the intermediary institutions, portfolio management companies, investment banks and the clients. Opinions and comments contained in this site reflect the personal views of the analysts / AI algorithms who supplied them. The investments discussed or recommended in this report may involve significant risk, may be illiquid and may not be suitable for all investors. Therefore, making decisions with respect to the information in this report may cause inappropriate results and a total loss.
Futures and forex trading contains substantial risk and is not for every investor. An investor could potentially lose all or more than the initial investment. Risk capital is money that can be lost without jeopardizing ones’ financial security or life style. Only risk capital should be used for trading and only those with sufficient risk capital should consider trading. Past performance is not necessarily indicative of future results
Hypothetical performance results have many inherent limitations, some of which are described below. no representation is being made that any account will or is likely to achieve profits or losses similar to those shown; in fact, there are frequently sharp differences between hypothetical performance results and the actual results subsequently achieved by any particular trading program. One of the limitations of hypothetical performance results is that they are generally prepared with the benefit of hindsight. In addition, hypothetical trading does not involve financial risk, and no hypothetical trading record can completely account for the impact of financial risk of actual trading. for example, the ability to withstand losses or to adhere to a particular trading program in spite of trading losses are material points which can also adversely affect actual trading results. There are numerous other factors related to the markets in general or to the implementation of any specific trading program which cannot be fully accounted for in the preparation of hypothetical performance results and all which can adversely affect trading results.
Testimonials appearing on this website may not be representative of other clients or customers and is not a guarantee of future performance or success.



No Investment Advice & No Reliance
YUCE-8 IS A DATA ANALYTICS SERVICE. YUCE-8 IS NO BROKER/DEALER, WE ARE NOT AN INVESTMENT ADVISOR, WE HAVE NO ACCESS TO NON-PUBLIC INFORMATION ABOUT PUBLICLY TRADED COMPANIES, AND THIS IS NOT A PLACE FOR THE GIVING OR RECEIVING OF FINANCIAL ADVICE, ADVICE CONCERNING INVESTMENT DECISIONS OR TAX OR LEGAL ADVICE. WE ARE NOT REGULATED BY THE SEC OR ANY FINANCIAL SERVICES AUTHORITY. 
ACCORDINGLY, YUCE-8 WILL NOT BE LIABLE, WHETHER IN CONTRACT, TORT (INCLUDING NEGLIGENCE) OR OTHERWISE, IN RESPECT OF ANY DAMAGE, EXPENSE OR OTHER LOSS YOU MAY SUFFER ARISING OUT OF SUCH INFORMATION OR ANY RELIANCE YOU MAY PLACE UPON SUCH INFORMATION. ANY ARRANGEMENTS BETWEEN YOU AND ANY THIRD PARTY CONTACTED VIA THE WEBSITE ARE AT YOUR SOLE RISK.